# Problem 1-3. Collage Collating

Fodoby is a company that makes customized software tools for creative people. Their newest
software, Ottoshop, helps users make collages by allowing them to overlay images on top of each
other in a single document. Describe a database to keep track of the images in a given document
which supports the following operations:

1. **make_document()**: construct an empty document containing no images
2. **import_image(x)**: add an image with unique integer ID x to the top of the document
3. **display()**: return an array of the document's image IDs in order from bottom to top
4. **move_below(x, y)**: move the image with ID x directly below the image with ID y

**Time Complexity Requirements:**
- Operation (1) should run in worst-case O(1) time
- Operations (2) and (3) should each run in worst-case O(n) time
- Operation (4) should run in worst-case O(log n) time

where n is the number of images contained in a document at the time of the operation.

----------------------

## Solution for Operations 1-3

### Data Structures:
We use two data structures working together:
1. **Doubly-linked list L**: Stores images in their actual order from bottom to top
2. **Sorted array S**: Stores pairs of (image_id, pointer_to_node_in_L), sorted by image_id for fast lookups

### Operations 1-3:

**1. make_document() - O(1)**
- Initialize L as an empty doubly-linked list
- Initialize S as an empty array
- Both operations take constant time

**2. import_image(x) - O(n)**
- Create a new node containing image x
- Add this node to the head of L (top of document) - O(1)
- Insert the pair (x, pointer_to_new_node) into S in sorted order - O(n)
  - This requires shifting existing elements to maintain sorted order
- Total: O(n) due to the array insertion

**3. display() - O(n)**
- Traverse L from tail to head (bottom to top)
- Collect all image IDs into an array and return it
- Must visit every node once: O(n)

------------------------------

# The move_below(x, y) Operation - Deep Dive

## The Challenge
We need to:
1. Find image x anywhere in the document
2. Find image y anywhere in the document  
3. Remove x from its current position
4. Place x directly below y
5. Do all this in O(log n) time

## The Solution Breakdown

### Step 1: Fast Image Lookup - O(log n)
```
Binary search in sorted array S for image x:
S = [(1, ptr1), (3, ptr3), (7, ptr7), (12, ptr12), (15, ptr15)]
Looking for image 7: binary search finds (7, ptr7) in O(log n)
```

**Why sorted array?** If we just had the linked list, we'd need O(n) time to scan through and find image x. The sorted array gives us O(log n) lookup.

### Step 2: Direct Access to Position - O(1)
Once we find `(7, ptr7)` in the array, `ptr7` directly points to the exact node in our linked list L where image 7 currently sits.

```
L: [img1] ↔ [img3] ↔ [img7] ↔ [img12] ↔ [img15]
                      ↑
                    ptr7 points here
```

**No scanning needed! We instantly know where image 7 is positioned.**

### Step 3: Efficient Removal - O(1)
With a **doubly-linked list**, removing a node when you have a direct pointer to it is O(1):

```
Before: [img3] ↔ [img7] ↔ [img12]
                  ↑
                ptr7

// Update pointers:
img3.next = img12
img12.prev = img3

After: [img3] ↔ [img12]
```

**Why doubly-linked?** With single links, we'd need to find the previous node, which takes O(n). Double links let us update both directions instantly.

### Step 4: Efficient Insertion - O(1)
Similarly, we use binary search to find image y in O(log n), get its pointer, then insert x below it in O(1):

```
Find y: Binary search finds (12, ptr12)
Insert x below y:

Before: [img3] ↔ [img12] ↔ [img15]
                  ↑
                ptr12

After:  [img3] ↔ [img12] ↔ [img7] ↔ [img15]
                          ↑
                    x inserted here
```

## Key Insight: Hybrid Data Structure Benefits

**Sorted Array S provides:**
- O(log n) lookup by image ID
- Direct pointers to linked list nodes

**Doubly-Linked List L provides:**
- O(1) removal when you have the pointer
- O(1) insertion when you know where to insert
- Natural ordering of images from bottom to top

## The Magic Moment
The crucial realization is that **we never need to update the sorted array S** during move_below! We're only changing the order in the linked list L, but each image keeps the same ID and the same pointer relationship. The array S still correctly maps image IDs to their nodes - those nodes just have different neighbors now.

This is why move_below runs in O(log n) instead of O(n) - we avoid the expensive array reorganization.

In [ ]:
class Node:
    """Simple node for doubly-linked list"""
    def __init__(self, image_id):
        self.image_id = image_id
        self.prev = None
        self.next = None

class CollageDatabase:
    """Simplified database for managing layered images"""
    
    def __init__(self):
        self.head = None  # Top of document (most recent)
        self.array = []   # Sorted array of (image_id, node) pairs
    
    def make_document(self):
        """O(1) - Create empty document"""
        self.head = None
        self.array = []
    
    def import_image(self, x):
        """O(n) - Add image to top"""
        # Create new node and add to top
        new_node = Node(x)
        if self.head:
            self.head.prev = new_node
            new_node.next = self.head
        self.head = new_node
        
        # Insert into sorted array - O(n)
        pos = 0
        while pos < len(self.array) and self.array[pos][0] < x:
            pos += 1
        self.array.insert(pos, (x, new_node))
    
    def display(self):
        """O(n) - Return images from bottom to top"""
        if not self.head:
            return []
        
        # Go to bottom (tail)
        current = self.head
        while current.next:
            current = current.next
        
        # Collect from bottom to top
        result = []
        while current:
            result.append(current.image_id)
            current = current.prev
        return result
    
    def move_below(self, x, y):
        """O(log n) - Move image x below image y"""
        # Binary search for both images - O(log n)
        x_node = self._find_node(x)
        y_node = self._find_node(y)
        
        if not x_node or not y_node:
            return  # Image not found
        
        # Remove x from current position - O(1)
        if x_node.prev:
            x_node.prev.next = x_node.next
        else:
            self.head = x_node.next
        
        if x_node.next:
            x_node.next.prev = x_node.prev
        
        # Insert x below y - O(1)
        x_node.prev = y_node
        x_node.next = y_node.next
        
        if y_node.next:
            y_node.next.prev = x_node
        y_node.next = x_node
    
    def _find_node(self, image_id):
        """Binary search to find node - O(log n)"""
        left, right = 0, len(self.array) - 1
        
        while left <= right:
            mid = (left + right) // 2
            if self.array[mid][0] == image_id:
                return self.array[mid][1]
            elif self.array[mid][0] < image_id:
                left = mid + 1
            else:
                right = mid - 1
        return None

# Simple test
db = CollageDatabase()
db.make_document()

# Add some images
db.import_image(5)
db.import_image(2)
db.import_image(8)
print("Initial:", db.display())  # [5, 2, 8]

# Move image 8 below image 5
db.move_below(8, 5)
print("After move:", db.display())  # [8, 5, 2]